# `2.0 Combined PDM 📊`

## <span style="color:tomato"> Overview </span>

In this notebook, we will compute the total power and total data for the configurations you created in the earlier notebooks. 

<span style="color:#18BF7D">If you need to make changes to your configurations, go back to the relevant notebooks, update the configurations, and re-run the cells with the lines "%store XXX".</span>

---

## <span style="color:tomato"> Calculating Additional Power Usage</span>

We have explored the power usage of many components on the LunaSat, but we have not considered all of them yet. Other components that consume power include the <span style="color:orange">LEDs</span> and the <span style="color:orange">EEPROM module</span>. Unfortunately the power is not very easy to predict for these components, and so the true power usage will have to be measured in the future. Additionally, the power consumption of the <span style="color:orange">EEPROM module</span> is dependent on how often data is read to and written to memory in the component, which is in turn dependent on the sampling rates you choose, the limitations imposed on those sampling rates, and the space available for data buffers in microcontroller memory. We will instead use an estimate of the power usage for now. In the cell below, you can choose how often the LEDs blink, if at all. LEDs may be used to support localization techniques, but they currently do not serve a purpose and will not be included in your chosen configurations. Another component that consumes power is the <span style="color:orange">microcontroller</span>. We will consider its power usage with the other sensors and with the RF system separately.

<span style="color:#18BF7D">For the following code, replace 'None' with appropriate values based on what you have learned above. Any lines without a 'None' should not be modified.</span>

In [ ]:
import micropip
await micropip.install("matplotlib")
import matplotlib.pyplot as plt
await micropip.install("numpy")
import numpy as np
from source.helperFunctions import *

voltage = 3.3 # Volts

min_pow = 29.7 #mW, minimum power always consumed by the LunaSat. Unavoidable.

eeprom_current_standby = 20 # µA
eeprom_current_io = 2 # mA
io_time = 4 #ns
io_per_second = None #times per second read/write is performed, value between 0 and 250 million

on_consumption = ((io_time/(10**6))*io_per_second)*(eeprom_current_io * (voltage * 1000))
off_consumption = (1-((io_time/(10**6))*io_per_second))*(eeprom_current_standby/1000 * (voltage * 1000))/1000
eeprom_power = on_consumption + off_consumption

led_blue_power = 66 # mW
led_red_power = 82.5 # mW
active_time_per_second_blue = None #seconds, value between 0 and 1
active_time_per_second_red = None #seconds, value between 0 and 1
led_power_consumption = led_blue_power * active_time_per_second_blue + led_red_power * active_time_per_second_red

add_power = eeprom_power + led_power_consumption

## <span style="color:tomato"> Creating Data Lists</span>

We want to now look at the power and data usages separetely before plotting them collectively. We'll first create Python lists that will include all individual power / data usages.

<span style="color:#18BF7D">

### Please read these instructions very carefully.
    
In order to correctly plot power and data usage for all components combined, you must revisit the earlier notebooks and change the durations of each configuration so that the duration of the first configuration for all components is the same (with the same going for the other configurations).
    
This is also a good opportunity to make sure that not only are the durations of the RF configurations adjusted as necessary, but also that RF is only on when all sensors are off, or are at a minimum not collecting data. You can keep these configurations as is for now just to see how the power usage of the RF system contributes to the total power usage before changing them.

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
# Retrieve the configuration data/power usage you generated in earlier notebooks
%store -r

min_power = min_pow * np.ones(len(acc_power))

min_time = tmp_time.copy()
min_data = [0] * tmp_data
add_data = [0] * tmp_data
add_time = tmp_time.copy()
total_time = tmp_time.copy()

total_power = tmp_power + mag_power + acc_power + tp_power + cap_power + mcr_power + add_power + min_power # computing total power for all sensors

total_data = tmp_data + mag_data + acc_data + tp_data + cap_data + mcr_data + add_data + min_data  # computing total data for all sensors

time_list = [min_time, acc_time, mag_time, tp_time, tmp_time, cap_time, mcr_time, total_time]
power_list = [min_power, acc_power, mag_power, tp_power, tmp_power, cap_power, mcr_power, total_power]
data_list = [min_data, acc_data, mag_data, tp_data, tmp_data, cap_data, mcr_data, total_data]

## <span style="color:tomato"> Putting It All Together </span>

We want to make sure that our configurations together meet power and data requirements. In this section, you should make sure that your configurations do not use more power than is available and do not generate more data than the maximum data rate. While these requirements have not been finalized, here are the parameters you should take into consideration:

- <span style="color:orange">**Landing Location (Latitude)**</span>: -70º
- <span style="color:orange">**Data rate**</span>: 1000 bytes per second
- <span style="color:orange">**Power Usage**</span>: Dependent on time of day. Minimum power needed is 29.7mW. Don't worry about satisfying this during lunar night or at the very beginning/end of lunar day.

The RF reception rate will add to the data rate of the LunaSat while the RF transmission rate will subtract from the data rate of the LunaSat.

<span style="color:#18BF7D">Your task should be to use the plots below to verify that your configurations meet these requirements. If you need to make changes to your configurations to meet these requirements, you can go back to the appropriate notebooks, adjust the configurations, and re-run the cells with the lines "%store XXX".</span>

<span style="color:#18BF7D">If you need to change the landing location of the LunaSats or the interval of time you'd like to look at for accurate plotting, go back to the Solar Panels notebook, alter the parameters passed into the SM111K object named "solar_panel_model", and re-run the cells with the lines "%store XXX".</span>

<span style="color:#18BF7D">Do not modify the following code. Simply run the cells to see the output.</span>

In [ ]:
plot_total_data(time_list, data_list)

In [ ]:
plot_total_power(time_list, power_list)

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output. The times at which your configurations can be run will be shown in blue under the curve. Note that if even one of your configurations exceeds the maximum available power, the configurations will not be able to be run at a particular time.</span>

In [ ]:
# The model shows the times in the shaded region where the solar panel produces enough power for the LunaSat with the given modes for each sensor
max_power = max(total_power) * np.ones(len(total_power))
solar_panel_model.plot_power_and_times_possible(max_power)

## <span style="color:tomato"> Optional: RF </span>

Under the assumption that RF will only be active while all other non-essential components are turned off, we can check to see if the power usage and data usage of the RF transceiver and microcontroller together also satisfy our requirements.

<span style="color:#18BF7D">Do not modify the following code. Simply run the cells to see the output. The times at which your configurations can be run will be shown in a dark blue. Note that if even one of your configurations exceeds the maximum available power, the configurations will not be able to be run.</span>

In [ ]:
rf_micro_power = mcr_power + add_power + rf_power + min_power # computing total power for all sensors

rf_micro_data = mcr_data + add_data + rf_data + min_data  # computing total data for all sensors

time_list = [min_time, mcr_time, rf_time, total_time]
power_list = [min_power, mcr_power, rf_power, total_power]
data_list = [min_data, mcr_data, rf_data, total_data]

plot_rf_data(time_list,data_list)

max_power = max(rf_micro_power) * np.ones(len(rf_micro_power))
solar_panel_model.plot_power_and_times_possible(max_power)

<span style="color:#18BF7D">Once you have determined that your configurations satisfy your use cases and the requirements, you can move on to the next notebook to generate bitstring representations of your configurations so that they can be included in the flight code.</span>

[Click here to continue to the bitstring generation notebook.](2.1%20Bitstring%20Generation.ipynb)